In [ ]:
pip install diffprivlib

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")
from myimports  import *

plt.style.use('ggplot')

In [ ]:
from aggregated_models.aggdataset import AggDataset
from validation import MetricsComputer


In [ ]:
from RandomProjectedPrivateLogistic import RPPrivateLogistic

In [ ]:
from aggLogistic import AggLogistic
from agg_mrf_model import AggMRFModel, AggMRFModelParams

In [ ]:
Validation = MetricsComputer("click")

## Download Data

In [ ]:
import loaddata
train, valid, allvars = loaddata.run()
## features def: https://confluence.criteois.com/pages/viewpage.action?spaceKey=RSC&title=Dataset+06+%3A+Attribution+Modeling+for+Bidding


In [ ]:
def addToResults( model, name , epsilon, regulL2 , otherparams = {}  ):
    dico = otherparams.copy()
    dico["model"] = name
    dico["valid"] = Validation.getLLH( model, valid )
    dico["train"] = Validation.getLLH( model, train )
    dico["epsilon"] = epsilon
    dico["regul"] = regulL2
    dico["features"] =  ",".join( model. features)
    res = pd.DataFrame( [dico] )
    print(res)
    return pd.concat( [ results , res ] )

In [ ]:
results = pd.DataFrame()

# Data set with limited number of features

In [ ]:
features = ['cat1', 'cat2', 'cat8', 'cat9' ]
aggdata_nonoise = AggDataset( features, "*&*", train )
epsilons = [ 0.1 ,1.0, 10.0 ]
aggdata_noised = {}
for epsilon0 in epsilons:
    aggdata_noised[epsilon0] = AggDataset( train , features, "*&*",  epsilon0=epsilon0)


In [ ]:
logistic.features

##### logistic Regression
- Requires full logs, not privacy preserving.
- "Skyline"

In [ ]:
regulL2s = [ 1.0,  4.0 , 16.0 , 64.0, 256 ]
for regulL2 in regulL2s:
    logistic = AggLogistic(  aggdata_nonoise , features, clicksCfs = "*&*" , regulL2=regulL2 )
    logistic.fit(train [features] , nbIter=500)
    results = addToResults( logistic, "logistic", 0, regulL2  )

In [ ]:
results

##### Private logistic regression
- From ibm diffprivlib
- Adding some noise to the loss
- Does not scales well with input vector dimensions, therefore using random projections to limit dimensionality

In [ ]:
%%time
for epsilon in epsilons:
    aggdata = aggdata_noised[epsilon]
    regulL2s = [ 1.0,  4.0 , 16.0 , 64.0, 256, 1024 , 1024*4 ]    
    for l2 in regulL2s:
        for p in [ 10,30,100,300, 1000 ]:
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

##### Models learned from agg data only

In [ ]:
%%time
for epsilon in epsilons:
    aggdata = aggdata_noised[epsilon]
    for l2 in [  4.0 , 16.0 , 64.0, 256, 1024 , 1024*4 ]:
        params = AggMRFModelParams(
            exactComputation=False,
            clicksCfs = "*&*",
            displaysCfs="*&*",
            nbSamples = 50000,
            regulL2=1.0,
            regulL2Click = l2,
            laplaceEpsilon = aggdata.epsilon
        )
        self = AggMRFModel(aggdata, features, params)
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

In [ ]:
results[results.epsilon == 1.0].sort_values( "valid" )

In [ ]:
a = results[(results.model == "aggmodel") & (results.epsilon == 0.1) &(results.train>0)]
a

In [ ]:
a = results[results.model == "logistic"]
plt.plot( a.train.values , a.valid.values  , "x" )


a = results[(results.model == "aggmodel") & (results.epsilon == 0.1) &(results.train>0)]
plt.plot( a.train.values , a.valid.values  , "x" )

a = results[(results.model == "privatelogistic") & (results.epsilon == 0.1) &(results.train>0)]
plt.plot( a.train.values , a.valid.values  , "kx" )



## Big dataset (11 features)

In [ ]:
features =['campaign', 'time_since_last_click', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
aggdata_nonoise = AggDataset( features, "*&*", train )
epsilons = [ 1.0, 10.0 ]

In [ ]:
aggdata_noised = {}
for epsilon0 in epsilons:
    aggdata_noised[epsilon0] = AggDataset( train , features, "*&*",  epsilon0=epsilon0)

In [ ]:
regulL2s = [  40.0 , 160.0 , 640.0, 2560 ]
for regulL2 in regulL2s:
    logistic = AggLogistic(  aggdata_nonoise , features, clicksCfs = "*&*" , regulL2=regulL2 )
    logistic.fit(train [features] , nbIter=50)
    results = addToResults( logistic, "logistic", 0, regulL2  )

In [ ]:
%%time
for epsilon in epsilons:
    regulL2s = [ 160.0 , 640.0, 2560, 2560*4 , 2560*16 ]    
    for l2 in regulL2s:
        for p in [ 30,100,300, 1000, 3000 ]:
            print( epsilon, l2,p)
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

In [ ]:
%%time
for epsilon in epsilons:
    regulL2s = [ 40.0, 10.0  ]    
    for l2 in regulL2s:
        for p in [ 3000,1000,300, 100, 30 ]:
            print( epsilon, l2,p)
            logistic = RPPrivateLogistic("click" , features, "*&*",  train, lambdaL2=l2,epsilon= epsilon,
                                     projectionSize=p )
            logistic.fit(train)
            results = addToResults( logistic, "privatelogistic",  epsilon , l2, {"dimensions":p}  )

In [ ]:
results.query( 'epsilon==10.0' ) .sort_values('valid')

In [ ]:
%%time
for epsilon in epsilons:
    aggdata = AggDataset( train , features, "*&*",  epsilon0=epsilon)
    for l2 in [ 160.0 , 640.0, 2560, 2560*4 , 2560*16 ]:
        params = AggMRFModelParams(
            exactComputation=False,
            clicksCfs = "*&*",
            displaysCfs="*&*",
            nbSamples = 50000,
            regulL2=1.0,
            regulL2Click = l2,
            noiseDistribution = aggdata.mechanism.getNoise()
        )
        self = AggMRFModel(aggdata, features, params)
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

In [ ]:
%%time
for epsilon in epsilons[1:]:
    aggdata = AggDataset( train , features, "*&*",  epsilon0=epsilon)
    for l2 in [ 160.0 , 640.0] :# , 2560, 2560*4 , 2560*16 ]:
        params = AggMRFModelParams(
            exactComputation=False,
            clicksCfs = "*&*",
            displaysCfs="*&*",
            nbSamples = 50000,
            regulL2=1.0,
            regulL2Click = l2,
            noiseDistribution = aggdata.mechanism.getNoise()
        )
        self = AggMRFModel(aggdata, features, params)
        self.fit(200)
        results = addToResults( self, "aggmodel",  epsilon , l2  )

In [ ]:
%%time
## GAUSSIAN noise
for delta in [ 0.000001 , 0.001]:
    for epsilon in [10.0, 1.0]:
        for l2 in [ 160.0 , 640.0] :# , 2560, 2560*4 , 2560*16 ]:
            aggdata = AggDataset( train , features, "*&*",  epsilon0=epsilon ,delta = delta)
            params = AggMRFModelParams(
                exactComputation=False,
                clicksCfs = "*&*",
                displaysCfs="*&*",
                nbSamples = 50000,
                regulL2=1.0,
                regulL2Click = l2,
                noiseDistribution = aggdata.mechanism.getNoise()
            )
            self = AggMRFModel(aggdata, features, params)
            self.fit(200)
            results = addToResults( self, "aggmodel",  epsilon , l2 ,  {"delta":delta}  )

In [ ]:
results[ (results.model=="privatelogistic") &(results.epsilon==0.1) ].sort_values('valid')

In [ ]:
results[results.epsilon==1.0].sort_values('valid')

In [ ]:
results[results.epsilon==10.0].sort_values('valid')